In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Data1.csv')

In [ ]:
import pandas as pd

# Assuming 'df' is your pandas DataFrame containing the dataset
# Assuming 'labels' is the column containing the labels

# Count the occurrences of each label type
label_counts = df['Label'].value_counts()

# Display the counts
print("Counts of each type of hate speech and normal comments:")
print(label_counts)


Counts of each type of hate speech and normal comments:
Label
hatespeech    640
normal        572
Name: count, dtype: int64


TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Load data
df = pd.read_csv('/content/Data1.csv')
X = df['Comment']
y = df['Label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)

# Fit and transform the training data, transform the testing data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Initialize the Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
random_forest.fit(X_train_tfidf, y_train)

# Predict on the test data
rf_pred = random_forest.predict(X_test_tfidf)

# Evaluate the classifier
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("\nRandom Forest Classification Report:\n", classification_report(y_test, rf_pred))


Random Forest Accuracy: 0.7736625514403292

Random Forest Classification Report:
               precision    recall  f1-score   support

  hatespeech       0.83      0.75      0.78       134
      normal       0.72      0.81      0.76       109

    accuracy                           0.77       243
   macro avg       0.77      0.78      0.77       243
weighted avg       0.78      0.77      0.77       243



In [ ]:
# Initialize the SVM classifier
svm_model = SVC(kernel='linear', random_state=42)  # 'linear' kernel is generally good for text

# Train the classifier
svm_model.fit(X_train_tfidf, y_train)

# Predict on the test data
svm_pred = svm_model.predict(X_test_tfidf)

# Evaluate the classifier
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("\nSVM Classification Report:\n", classification_report(y_test, svm_pred))


SVM Accuracy: 0.8024691358024691

SVM Classification Report:
               precision    recall  f1-score   support

  hatespeech       0.82      0.82      0.82       134
      normal       0.78      0.78      0.78       109

    accuracy                           0.80       243
   macro avg       0.80      0.80      0.80       243
weighted avg       0.80      0.80      0.80       243



WORD2VEC

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import gensim
from gensim.models import Word2Vec
import numpy as np
# Load data
df = pd.read_csv('/content/Data1.csv')
X = df['Comment'].apply(gensim.utils.simple_preprocess)  # Tokenize text
y = df['Label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Initialize and train a Word2Vec model
model_w2v = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)

# Use the trained model to transform words to vectors and then average those vectors to create a document vector
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.index_to_key]
    if len(doc) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc], axis=0)

X_train_w2v = np.array([document_vector(model_w2v, words) for words in X_train])
X_test_w2v = np.array([document_vector(model_w2v, words) for words in X_test])


In [ ]:
# Initialize the Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
random_forest.fit(X_train_w2v, y_train)

# Predict on the test data
rf_pred = random_forest.predict(X_test_w2v)

# Evaluate the classifier
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("\nRandom Forest Classification Report:\n", classification_report(y_test, rf_pred))


Random Forest Accuracy: 0.6378600823045267

Random Forest Classification Report:
               precision    recall  f1-score   support

  hatespeech       0.66      0.70      0.68       134
      normal       0.60      0.56      0.58       109

    accuracy                           0.64       243
   macro avg       0.63      0.63      0.63       243
weighted avg       0.64      0.64      0.64       243



In [ ]:
# Initialize the SVM classifier
svm_model = SVC(kernel='linear', random_state=42)  # 'linear' kernel is generally good for text

# Train the classifier
svm_model.fit(X_train_w2v, y_train)

# Predict on the test data
svm_pred = svm_model.predict(X_test_w2v)

# Evaluate the classifier
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("\nSVM Classification Report:\n", classification_report(y_test, svm_pred))


SVM Accuracy: 0.551440329218107

SVM Classification Report:
               precision    recall  f1-score   support

  hatespeech       0.55      1.00      0.71       134
      normal       0.00      0.00      0.00       109

    accuracy                           0.55       243
   macro avg       0.28      0.50      0.36       243
weighted avg       0.30      0.55      0.39       243



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
